In [1]:
import pandas as pd
import numpy as np
import requests, bs4
import re, os
import matplotlib.pyplot as plt
import statsapi
from datetime import date
import time


In [2]:
# Scrape information (players with ACTIVE hit streaks of >= 5 games) from baseballmusings.com)

today = date.today()
today = str(today)[-2:]
today

url = f'https://www.baseballmusings.com/cgi-bin/CurStreak.py?EndDate=08%2F{today}%2F2022'
resp = requests.get(url)

with open('test_mlb_longesthitstreak', 'wb') as f:
    f.write(resp.content)

df_streaks = pd.read_html('test_mlb_longesthitstreak')
df1 = pd.DataFrame(df_streaks[1])
df1 = df1.loc[(df1['Player'] != 'Player')]
df1

,Player,Games,At Bats,Runs,Hits,HR,RBI,BB,K,BA,OBA,Slug%,Last Game Date
0,Adolis Garcia,16,63,9,20,1,11,3,17,.317,.368,.413,2022-08-19
1,Kyle Tucker,12,49,11,20,3,16,3,5,.408,.442,.735,2022-08-19
2,Manny Machado,11,47,9,20,3,16,4,7,.426,.471,.787,2022-08-19
3,Luis Robert,10,42,7,18,2,10,4,5,.429,.478,.667,2022-08-17
4,Franmil Reyes,9,38,5,14,2,6,0,10,.368,.368,.711,2022-08-19
5,Eloy Jimenez,9,31,4,12,0,4,6,7,.387,.486,.484,2022-08-19
6,William Contreras,9,38,5,12,1,3,2,8,.316,.350,.447,2022-08-17
7,Nick Castellanos,8,34,4,12,1,2,0,13,.353,.353,.529,2022-08-19
8,Cesar Hernandez,7,30,4,11,0,1,0,5,.367,.367,.433,2022-08-19
9,Francisco Mejia,7,27,3,10,0,4,1,3,.370,.414,.519,2022-08-19


In [9]:
def get_todays_hitstreak_info(df):
    todays_info = []
    for name in df['Player']:
        team_id = statsapi.lookup_player(name)[0]['currentTeam']['id']
        full_team_name = statsapi.lookup_team(team_id)[0]['name']
        #team_abbrv = statsapi.lookup_team(team_id)[0]['teamCode']
        #team_name = statsapi.lookup_team(team_id)[0]['teamName']
        player_info = (name, full_team_name, team_id)
        todays_info.append(player_info)
        time.sleep(2)
    print(pd.DataFrame(todays_info, columns = ['Player', 'Full Team Name', 'Team ID']).sort_values(by = 'Full Team Name'))
        
get_todays_hitstreak_info(df1)

               Player         Full Team Name  Team ID
8     Cesar Hernandez   Washington Nationals      120
20        Lane Thomas   Washington Nationals      120
14    George Springer      Toronto Blue Jays      141
0       Adolis Garcia          Texas Rangers      140
11     Bubba Thompson          Texas Rangers      140
10          Josh Lowe         Tampa Bay Rays      139
9     Francisco Mejia         Tampa Bay Rays      139
17     Harold Ramirez         Tampa Bay Rays      139
13       Nolan Gorman    St. Louis Cardinals      138
22   Jake Cronenworth       San Diego Padres      135
2       Manny Machado       San Diego Padres      135
15       Kevin Newman     Pittsburgh Pirates      134
7    Nick Castellanos  Philadelphia Phillies      143
19     Garrett Stubbs  Philadelphia Phillies      143
21     Luis Guillorme          New York Mets      121
16       Ryan Jeffers        Minnesota Twins      142
1         Kyle Tucker         Houston Astros      117
18       Victor Reyes       

In [8]:
team_dict = {
    108: ('LAA', 'Angels', '#GoHalos'),
    109: ('ARI', 'D-backs', '#Dbacks'),
    110: ('BAL', 'Orioles', '#Birdland'),
    111: ('BOS', 'Red Sox', '#DirtyWater'),
    112: ('CHC', 'Cubs', '#ItsDifferentHere'),
    113: ('CIN', 'Reds', '#ATOBTTR'),
    114: ('CLE', 'Guardians', '#ForTheLand'),
    115: ('COL', 'Rockies', '#Rockies'),
    116: ('DET', 'Tigers', '#DetroitRoots'),
    117: ('HOU', 'Astros', '#LevelUp'),
    118: ('KC', 'Royals', '#TogetherRoyal'),
    119: ('LAD', 'Dodgers', '#AlwaysLA'),
    120: ('WSH', 'Nationals', '#NATITUDE'),
    121: ('NYM', 'Mets', '#LGM'),
    133: ('OAK', 'Athletics', '#DrumTogether'),
    134: ('PIT', 'Pirates', '#LetsGoBucs'),
    135: ('SD', 'Padres', '#TimeToShine'),
    136: ('SEA', 'Mariners', '#SeaUsRise'),
    137: ('SF', 'Giants', '#SFGameUp'),
    138: ('STL', 'Cardinals', '#STLCards'),
    139: ('TB', 'Rays', '#RaysUp'),
    140: ('TEX', 'Rangers', '#StraightUpTX'),
    141: ('TOR', 'Blue Jays', '#NextLevel'),
    142: ('MIN', 'Twins', '#MNTwins'),
    143: ('PHI', 'Phillies', '#RingTheBell'),
    144: ('ATL', 'Braves', '#ForTheA'),
    145: ('CWS', 'White Sox', '#ChangeTheGame'),
    146: ('MIA', 'Marlins', '#MakeItMiami'),
    147: ('NYY', 'Yankees', '#RepBX'),
    158: ('MIL', 'Brewers', '#ThisIsMyCrew')
}

In [ ]:
pd.DataFrame([('Austin Riley', 144, 'Atlanta Braves', 'atl', 'Braves'), ('Francisco Lindor', 121, 'New York Mets', 'nyn', 'Mets'), ('Corey Seager', 140, 'Texas Rangers', 'tex', 'Rangers'), ('Carson Kelly', 109, 'Arizona Diamondbacks', 'ari', 'D-backs'), ('Brandon Drury', 113, 'Cincinnati Reds', 'cin', 'Reds'), ('Freddie Freeman', 119, 'Los Angeles Dodgers', 'lan', 'Dodgers'), ('Leody Taveras', 140, 'Texas Rangers', 'tex', 'Rangers'), ('Charlie Blackmon', 115, 'Colorado Rockies', 'col', 'Rockies'), ('Jose Abreu', 145, 'Chicago White Sox', 'cha', 'White Sox'), ("Ke'Bryan Hayes", 134, 'Pittsburgh Pirates', 'pit', 'Pirates'), ('Amed Rosario', 114, 'Cleveland Guardians', 'cle', 'Guardians'), ('Adam Frazier', 136, 'Seattle Mariners', 'sea', 'Mariners'), ('Seiya Suzuki', 112, 'Chicago Cubs', 'chn', 'Cubs'), ('Trea Turner', 119, 'Los Angeles Dodgers', 'lan', 'Dodgers'), ('Jonathan India', 113, 'Cincinnati Reds', 'cin', 'Reds'), ('Nick Castellanos', 143, 'Philadelphia Phillies', 'phi', 'Phillies'), ('Isiah Kiner-Falefa', 147, 'New York Yankees', 'nya', 'Yankees'), ('Alex Kirilloff', 142, 'Minnesota Twins', 'min', 'Twins'), ('Garrett Hampson', 115, 'Colorado Rockies', 'col', 'Rockies'), ('Lourdes Gurriel', 141, 'Toronto Blue Jays', 'tor', 'Blue Jays'), ('Alejandro Kirk', 141, 'Toronto Blue Jays', 'tor', 'Blue Jays'), ('Brendan Rodgers', 115, 'Colorado Rockies', 'col', 'Rockies'), ('Tim Anderson', 145, 'Chicago White Sox', 'cha', 'White Sox'), ('Matt Olson', 144, 'Atlanta Braves', 'atl', 'Braves'), ('Alec Bohm', 143, 'Philadelphia Phillies', 'phi', 'Phillies'), ('Austin Nola', 135, 'San Diego Padres', 'sdn', 'Padres'), ('Lars Nootbaar', 138, 'St. Louis Cardinals', 'sln', 'Cardinals'), ('Nelson Cruz', 120, 'Washington Nationals', 'was', 'Nationals'), ('Luke Williams', 146, 'Miami Marlins', 'mia', 'Marlins'), ('Yoan Moncada', 145, 'Chicago White Sox', 'cha', 'White Sox'), ('Ryan Jeffers', 142, 'Minnesota Twins', 'min', 'Twins'), ('Harold Castro', 116, 'Detroit Tigers', 'det', 'Tigers'), ('Taylor Walls', 139, 'Tampa Bay Rays', 'tba', 'Rays'), ('Skye Bolt', 133, 'Oakland Athletics', 'oak', 'Athletics'), ('Ryan McMahon', 115, 'Colorado Rockies', 'col', 'Rockies'), ('C.J. Cron', 115, 'Colorado Rockies', 'col', 'Rockies'), ('Yasmani Grandal', 145, 'Chicago White Sox', 'cha', 'White Sox'), ('Enrique Hernandez', 111, 'Boston Red Sox', 'bos', 'Red Sox'), ('Joey Votto', 113, 'Cincinnati Reds', 'cin', 'Reds'), ('Nicky Lopez', 118, 'Kansas City Royals', 'kca', 'Royals')])

,0,1,2,3,4
0,Austin Riley,144,Atlanta Braves,atl,Braves
1,Francisco Lindor,121,New York Mets,nyn,Mets
2,Corey Seager,140,Texas Rangers,tex,Rangers
3,Carson Kelly,109,Arizona Diamondbacks,ari,D-backs
4,Brandon Drury,113,Cincinnati Reds,cin,Reds
5,Freddie Freeman,119,Los Angeles Dodgers,lan,Dodgers
6,Leody Taveras,140,Texas Rangers,tex,Rangers
7,Charlie Blackmon,115,Colorado Rockies,col,Rockies
8,Jose Abreu,145,Chicago White Sox,cha,White Sox
9,Ke'Bryan Hayes,134,Pittsburgh Pirates,pit,Pirates
